# Keras exercise

In this exercise you will be creating a Keras model by loading a data set, preprocessing input data, building a Sequential Keras model and compiling the model with a training configuration. Afterwards, you train your model on the training data and evaluate it on the test set. To finish this exercise, you will then save your model in HDF5 format.

## Data

For this exercise we will use the Reuters newswire dataset. This dataset consists of 11,228 newswires from the Reuters news agency. Each wire is encoded as a sequence of word indexes, just as in the IMDB data we encountered in lecture 5 of this series. Moreover, each wire is categorised into one of 46 topics, which will serve as our label. This dataset is available through the Keras API.

## Goal

We want to create a Multi-layer perceptron (MLP) using Keras which we can train to classify news items into the specified 46 topics.

## Instructions

We start by importing everything we need for this exercise:

In [5]:
import pip

try:
    __import__('keras')
except ImportError:
    pip.main(['install', 'keras']) 
    
try:
    __import__('h5py')
except ImportError:
    pip.main(['install', 'h5py']) 

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

seed = 1337
np.random.seed(seed)

IMPORTANT! => In case h5py has been installed, please restart the kernel by clicking on "Kernel"->"Restart and Clear Outout" and wait until all output disapears. Then your changes are beeing picked up

As you can see, we use Keras' Sequential model with only two types of layers: Dense and Dropout. We also specify a random seed to make our results reproducible. Next, we load the Reuters data set:

In [6]:
from keras.datasets import reuters

max_words = 1000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,test_split=0.2, seed=seed)
num_classes = np.max(y_train) + 1  # 46 topics

Note that we cap the maximum number of words in a news item to 1000 by specifying the *num_words* key word. Also, 20% of the data will be test data and we ensure reproducibility by setting our random seed.

Our training features are still simply sequences of indexes and we need to further preprocess them, so that we can plug them into a *Dense* layer. For this we use a *Tokenizer* from Keras' text preprocessing module. This tokenizer will take an index sequence and map it to a vector of length *max_words=1000*. Each of the 1000 vector positions corresponds to one of the words in our newswire corpus. The output of the tokenizer has a 1 at the i-th position of the vector, if the word corresponding to i is in the description of the newswire, and 0 otherwise. Even if this word appears multiple times, we still just put a 1 into our vector, i.e. our tokenizer is binary. We use this tokenizer to transform both train and test features:

In [7]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

## 1. Exercise part: label encoding

Use to_categorical, as we did in the lectures, to transform both *y_train* and *y_test* into one-hot encoded vectors of length *num_classes*:

In [8]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

## 2. Exercise part: model definition

Next, initialise a Keras *Sequential* model and add three layers to it:

    Layer: Add a *Dense* layer with in input_shape=(max_words,), 512 output units and "relu" activation.
    Layer: Add a *Dropout* layer with dropout rate of 50%.
    Layer: Add a *Dense* layer with num_classes output units and "softmax" activation.

In [20]:
model = Sequential()###_YOUR_CODE_GOES_HERE_###  # Instantiate sequential model
#model.add(Dense(512, activation='relu',input_shape=(max_words,)))
model.add(Dense(512, activation='relu', input_shape=(max_words,))) # Add first layer. Make sure to specify input shape
model.add(Dropout(0.5)) ## Add second layer
model.add(Dense(num_classes, activation='softmax')) # Add third layer

## 3. Exercise part: model compilation

As the next step, we need to compile our Keras model with a training configuration. Compile your model with "categorical_crossentropy" as loss function, "adam" as optimizer and specify "accuracy" as evaluation metric.

In [21]:
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

## 4. Exercise part: model training and evaluation

Next, define the batch_size for training as 32 and train the model for 5 epochs on *x_train* and *y_train* by using the *fit* method of your model. Then calculate the score for your trained model by running *evaluate* on *x_test* and *y_test* with the same batch size as used in *fit*.

In [22]:
batch_size =32
model.fit(x_train,y_train, batch_size=batch_size, epochs=5, validation_data=(x_test,y_test))
score = model.evaluate(x_test,y_test, verbose=0)

Train on 8982 samples, validate on 2246 samples
Epoch 1/5
8982/8982 [==============================] - 7s - loss: 1.3914 - acc: 0.6872 - val_loss: 0.9913 - val_acc: 0.7734
Epoch 2/5
8982/8982 [==============================] - 6s - loss: 0.7681 - acc: 0.8210 - val_loss: 0.8365 - val_acc: 0.8045
Epoch 3/5
8982/8982 [==============================] - 6s - loss: 0.5515 - acc: 0.8666 - val_loss: 0.8177 - val_acc: 0.8001
Epoch 4/5
8982/8982 [==============================] - 6s - loss: 0.4114 - acc: 0.8977 - val_loss: 0.8185 - val_acc: 0.8019
Epoch 5/5
8982/8982 [==============================] - 6s - loss: 0.3408 - acc: 0.9109 - val_loss: 0.8473 - val_acc: 0.8059


If you have done everything as specified, in particular set the random seed as we did above, your test accuracy should be around 80% 

In [23]:
score[1]

0.80587711492395786



## 5. Exercise part: model serialisation

As the final step in this exercise, save your model as "model.h5" and upload this file for evaluation:

In [24]:
model.save("model.h5")  # upload this file to the grader in the next code block

To upload the exported model to the grader we first need to encode it based64, we are doing this using a shell command:

In [25]:
!base64 model.h5 > model.h5.base64

Then we have to install a little library in order to submit to coursera

In [26]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2018-03-17 19:02:16--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2029 (2.0K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,029       --.-K/s   in 0s      

2018-03-17 19:02:16 (12.6 MB/s) - ‘rklib.py’ saved [2029/2029]



Please provide your email address and obtain a submission token (secret) on the grader’s submission page in coursera, then execute the cell

In [29]:
from rklib import submit
key = "XbAMqtjdEeepUgo7OOVwng"
part = "LqPRQ"
email = "ashish222322@gmail.com"
secret = "W1EK2lvE4pxHMEd7"

with open('model.h5.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
